

# In this notebook, we will be going through the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for our agents, we'll be using OpenAI's `gpt-3.5-turbo`.


In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create a crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

- Display the results of your execution as markdown in the notebook.

In [13]:
topic = "Tesla stock price prediction"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Tesla stock price prediction.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Predicting the Future: An In-depth Analysis of Tesla Stock Price

Introduction:
- Brief overview of Tesla's history and its impact on the stock market
- Introduction to the topic of stock price prediction and the importance of making informed decisions

Key Points:
1. Latest Trends:
- Analysis of recent trends in Tesla's stock price movement
- Comparison with industry trends and market predictions
- Insights from financial experts and analysts on the future outlook

2. Key Players:
- Overview of 

I now can give a great answer

Final Answer:

# Predicting the Future: An In-depth Analysis of Tesla Stock Price

Tesla, a company founded by the visionary Elon Musk, has transformed the automotive industry and beyond with its cutting-edge electric vehicles and sustainable energy solutions. The company's remarkable success and its impact on the stock market have attracted widespread attention from investors and analysts alike. Predicting Tesla's stock price is a critical aspect of making informed investment decisions and maximizing returns in a highly volatile market. In this blog post, we aim to explore the realm of Tesla stock price prediction by examining trends, key players, and significant news events to offer valuable insights for individual investors, financial professionals, and enthusiasts of the stock market.

Recent movements in Tesla's stock price have been a focal point for many market participants. The stock has exhibited notable volatility, influenced by various factors 

In [14]:
Markdown(result)

# Predicting the Future: An In-depth Analysis of Tesla Stock Price

Tesla, a company founded by the visionary Elon Musk, has transformed the automotive industry and beyond with its cutting-edge electric vehicles and sustainable energy solutions. The company's remarkable success and its impact on the stock market have attracted widespread attention from investors and analysts alike. Predicting Tesla's stock price is a critical aspect of making informed investment decisions and maximizing returns in a highly volatile market. In this blog post, we aim to explore the realm of Tesla stock price prediction by examining trends, key players, and significant news events to offer valuable insights for individual investors, financial professionals, and enthusiasts of the stock market.

Recent movements in Tesla's stock price have been a focal point for many market participants. The stock has exhibited notable volatility, influenced by various factors such as production figures, delivery statistics, and the overall market sentiment towards electric vehicle manufacturers. Financial experts and analysts have been meticulously observing these trends, drawing comparisons with industry data and market forecasts to predict the future trajectory of Tesla's stock price. By scrutinizing the latest data and perspectives from credible sources like Bloomberg and CNBC, investors can develop a deeper understanding of the forces driving Tesla's stock price fluctuations.

Elon Musk, the enigmatic CEO of Tesla, wields considerable influence over the company's stock price through his public statements, decisions, and actions. Institutional investors and market influencers also hold significant sway over Tesla's stock price, as their trading activities can impact market sentiment and drive price movements. Grasping the dynamics of these key players and their interactions within the stock market is crucial for accurately forecasting Tesla's stock price. By examining insights from reputable sources such as Forbes and staying abreast of financial news, investors can remain informed about the latest developments and make well-grounded choices regarding their Tesla stock investments.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).